In [1]:
import qutip as qt
import numpy as np
from scipy import sparse as sp

from trajectree.fock_optics.noise_models import *
from trajectree.fock_optics.measurement import *
from trajectree.fock_optics.utils import *
from trajectree.fock_optics.light_sources import *

from trajectree.trajectory import *

from trajectree.experimental.sparse import *
from trajectree.protocols.swap import *


/home/asingal/home/.venv/lib/python3.10/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
/home/asingal/home/.venv/lib/python3.10/site-packages/cotengra/hyperoptimizers/hyper.py:33: UserWarning: Couldn't import `kahypar` - skipping from default hyper optimizer and using basic `labels` method instead.
  warnings.warn(


In [2]:
# Set simulation params
trunc = 2
N = trunc+1
error_tolerance = 1e-15

mean_photon_num = 0.5 # (0.5)
det_eff = 1 # 0.9
channel_loss = 0.5


num_modes = 8
# Create vacuum state

idler_angles = np.linspace(0, np.pi, 1)
signal_angles = np.linspace(0, 4*np.pi, 30)

num_simulations = 256

cache_sizes = [2]

## Verifying Density matrix

In [3]:
# Find density matrix:
psi = create_swapping_initial_state(num_modes, N, mean_photon_num, error_tolerance)
# read_quantum_state(psi, N)
quantum_channels = generate_swapping_circuit(N, num_modes, psi.site_tags, det_eff, channel_loss, error_tolerance)
t_eval = trajectory_evaluator(quantum_channels)
dm = t_eval.calculate_density_matrix(psi, error_tolerance)

psi_minus = create_bimode_bell_state('psi_minus', N)

querying nodes: [0, 0]
querying nodes: [0, 1]
querying nodes: [0, 2]
querying nodes: [1, 0]
querying nodes: [1, 1]
querying nodes: [1, 2]
querying nodes: [2, 0]
querying nodes: [2, 1]
querying nodes: [2, 2]


In [4]:
a = qt.destroy(trunc+1).full()
a = sp.csr_matrix(a)
a_dag = a.T
    
N = trunc+1
vacuum = create_vacuum_state(num_modes=num_modes, N=N)

psi, TMSV_state = light_source(vacuum, N, mean_photon_num, num_modes, error_tolerance, compress=True, contract=True)

# Sparse matrix data:
TMSV_state_dense = extend_state_sparse(TMSV_state) # TMSV_state was created earlier. 

TMSV_dm = TMSV_state_dense @ TMSV_state_dense.conj().T

single_mode_kraus_ops = single_mode_bosonic_noise_channels(noise_parameter = channel_loss, N = N)
damping_kraus_ops = [sp.kron(op, op) for op in single_mode_kraus_ops]
damping_kraus_ops_1 = [create_op(2, op, 4, N) for op in damping_kraus_ops]
damping_kraus_ops_2 = [create_op(6, op, 0, N) for op in damping_kraus_ops]

new_dm = 0
for kraus_op in damping_kraus_ops_1:
    new_dm += kraus_op @ TMSV_dm @ kraus_op.conj().T

old_dm = new_dm
new_dm = 0
for kraus_op in damping_kraus_ops_2:
    new_dm += kraus_op @ old_dm @ kraus_op.conj().T

post_BSM_dm = bell_state_measurement_sparse(new_dm, N, det_eff, a_dag, is_dm = True)


(6561, 6561) 4374 4374


In [5]:
TN_dm_sparse = sp.csr_array(np.round(dm, 8))
# TN_dm_sparse.trace()
# print(TN_dm_sparse)

# print("dense version:")
post_BSM_dm.data = np.round(post_BSM_dm.data, 8)
post_BSM_dm.eliminate_zeros()
# print(post_BSM_dm)
# dense_dm_sparse.trace()

diff = TN_dm_sparse - post_BSM_dm
diff.data = np.round(diff.data, 8)
diff.eliminate_zeros()
diff

<6561x6561 sparse array of type '<class 'numpy.complex128'>'
	with 0 stored elements in Compressed Sparse Row format>